In [34]:
import pandas as pd
from utils import unpicklify
#df = pd.read_csv('logs/tf/mood_classification/moods_and_lyrics.csv')
df = unpicklify('logs/tf/mood_classification/moods_and_lyrics.csv')

In [35]:
import warnings
warnings.filterwarnings('ignore')

# Filtering

### Duplicates

(there are none because the moods_and_lyrics csv file already dropped duplicates)

### NAs

There are NAs in the non processed lyrics series. Thankfully, preprocessing removes the NAs.

In [73]:
print('na={0}'.format(df.lyrics.isna().sum()))
print('na pre={0}'.format(df.preprocessed_lyrics.isna().sum()))

na=497
na pre=0


### Instrumentals

Some songs in the datasets are instrumentals but still have lyrics. This can be a short 1 sentence introduction, the name of the artist and the song, or something else very brief.

The goal of this section is to find what value we should use as a minimum wordcount in our dataset.

#### Number of Instrumentals

In [74]:
def count_instrumentals(lyrics):
    lyrics = lyrics.dropna()
    return lyrics.str.lower().str.contains("instrumental").sum()

print('instrumentals={0}'.format(count_instrumentals(df.lyrics)))
print('instrumentals preprocessed={0}'.format(count_instrumentals(df.preprocessed_lyrics)))

instrumentals=764
instrumentals preprocessed=748


#### Examples of Instrumental "Lyrics"


In [77]:
def show_examples_based_on_wordcount(data, bottom_limit=None, top_limit=None):
    exdata = data.copy()
    if top_limit:
        exdata = exdata[exdata.wordcount < top_limit]
    if bottom_limit:
        exdata = exdata[exdata.wordcount > bottom_limit]
    count = 0
    limit = 10
    skip = 8
    print('*************************************************')
    print('{0}:{1}'.format(bottom_limit, top_limit))
    for index, x in exdata.iterrows():
        # don't just look at the beginning
        if index % skip == 0:
            print('-----------------------------')
            print('Example {0}, Track: {1}, Mood: {2}'.format(index, x['lyrics_filename'], x['mood']))
            print('Lyrics: {0}'.format(x['lyrics']))
            count += 1
            if count == limit:
                print('done')
                break


In [78]:
show_examples_based_on_wordcount(df, top_limit=10)
print('\n\n\n')
show_examples_based_on_wordcount(df, bottom_limit=10, top_limit=15)
print('\n\n\n')
show_examples_based_on_wordcount(df, bottom_limit=15, top_limit=20)

*************************************************
None:10
-----------------------------
Example 0, Track: _And_Oceans___New_Model_World, Mood: calm
Lyrics: And Oceans
Miscellaneous
New Model World
[instrumental]
-----------------------------
Example 408, Track: ATB___Underwater_World, Mood: calm
Lyrics: Won't you have some of that
Some of that
-----------------------------
Example 920, Track: Aim___Cold_Water_Music, Mood: calm
Lyrics: You know we got a great thing here
-----------------------------
Example 1176, Track: Alan_Parsons___Dreamscape, Mood: calm
Lyrics: Alan Parsons Project
Try Anything Once
Dreamscape
Instrumental
[parsons]
-----------------------------
Example 4776, Track: Beastie_Boys___Transitions, Mood: calm
Lyrics: Beastie Boys/Nishita)
(Instrumental
-----------------------------
Example 5376, Track: Betty_Davis___Special_People, Mood: sad
Lyrics: *Needs lyrics*
-----------------------------
Example 7192, Track: Bonobo___Kota, Mood: calm
Lyrics: [No Lyrics]
-----------

#### What is the proper minimum length?